# Subduction in the Aleutians

I was inspired by HaimFeldman to look closely at the earthquake data for Alaska. Much of the code here is borrowed from his excellent notebook on the earthquak and volcano data.

In particular, I was grabbed by the [Aleutian subduction zone](https://en.wikipedia.org/wiki/Aleutian_Trench), so I tried to measure the subduction angle from the data provided. This is still a work in progress and many improvements can be made. Take a look at the analysis and let me know what you think!

(Data are from United States Geological Survey and the Smithsonian.)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
from pandas import Series
print(check_output(["ls", "../input"]).decode("utf8"))


volcanoes = pd.read_csv("../input/volcanic-eruptions/database.csv")
earthquakes = pd.read_csv("../input/earthquake-database/database.csv")

In [2]:
#Defining Latitude and Longitude limits (change to your favorite part of the world!)
llcrnrlat = 45
urcrnrlat = 70
llcrnrlon = -180
urcrnrlon = -125

#Cutting out the data from other parts of the world.
alaska_volcanoes = volcanoes[(volcanoes.Longitude > llcrnrlon) & 
                             (volcanoes.Longitude < urcrnrlon) & 
                             (volcanoes.Latitude > llcrnrlat) & 
                             (volcanoes.Latitude < urcrnrlat)]

alaska_earthquakes = earthquakes[(earthquakes.Longitude > llcrnrlon) & 
                             (earthquakes.Longitude < urcrnrlon) & 
                             (earthquakes.Latitude > llcrnrlat) & 
                             (earthquakes.Latitude < urcrnrlat)]

In [3]:
m = Basemap(projection='mill',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat, llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,lat_ts=20,resolution='l')
fig = plt.figure(figsize=(12,10))


longitudes_vol = alaska_volcanoes.Longitude.tolist()
latitudes_vol = alaska_volcanoes.Latitude.tolist()

longitudes_eq = alaska_earthquakes.Longitude.tolist()
latitudes_eq = alaska_earthquakes.Latitude.tolist()

x,y = m(longitudes_vol,latitudes_vol)
a,b= m(longitudes_eq,latitudes_eq)

plt.title("Alaskan Volcanoes (red) Earthquakes (green)")
m.plot(x, y, "o", markersize = 5, color = 'red')
m.plot(a, b, "o", markersize = 3, color = 'green')

m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='coral',lake_color='aqua')
m.drawmapboundary()
m.drawcountries()
plt.show()

There's a really strong chain of volcanoes going out the Aleutian Islands, with a large set of recorded earthquakes just to the south and east of these volcanoes. 

In [4]:
llcrnrlat = 48
urcrnrlat = 61
llcrnrlon = -180
urcrnrlon = -150

#A closer look at the Aleutian Islands
aleutian_volcanoes = volcanoes[(volcanoes.Longitude > llcrnrlon) & 
                             (volcanoes.Longitude < urcrnrlon) & 
                             (volcanoes.Latitude > llcrnrlat) & 
                             (volcanoes.Latitude < urcrnrlat)]

aleutian_earthquakes = earthquakes[(earthquakes.Longitude > llcrnrlon) & 
                             (earthquakes.Longitude < urcrnrlon) & 
                             (earthquakes.Latitude > llcrnrlat) & 
                             (earthquakes.Latitude < urcrnrlat)]

In [5]:
m = Basemap(projection='mill',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat, llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,lat_ts=20,resolution='l')
fig = plt.figure(figsize=(12,10))


longitudes_vol = aleutian_volcanoes.Longitude.tolist()
latitudes_vol = aleutian_volcanoes.Latitude.tolist()

longitudes_eq = aleutian_earthquakes.Longitude.tolist()
latitudes_eq = aleutian_earthquakes.Latitude.tolist()

x,y = m(longitudes_vol,latitudes_vol)
a,b= m(longitudes_eq,latitudes_eq)

plt.title("Aleutian Volcanoes (red) Earthquakes (green)")
m.plot(x, y, "o", markersize = 5, color = 'red')
m.plot(a, b, "o", markersize = 3, color = 'green')

m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='coral',lake_color='aqua')
m.drawmapboundary()
m.drawcountries()
plt.show()

Since the Pacific Plate is dipping under the North American Plate, we may be able to see some relationship between the depth and position. The volcanoes on the chain are caused by the subduction, so we can relate the position of earthquakes to the position of the volcanoes. In this case, we'll record the distance to the closest volcano.

In [6]:
from geopy.distance import great_circle

In [7]:
volcano_locations = aleutian_volcanoes[['Latitude', 'Longitude']]

def d_to_volcano(row):
    '''
    Given a row of the earthquakes DataFrame, returns the distance to the nearest aleutian volcano.
    
    There has got to be a better (faster) way to do this.
    '''
    min_dist = 1e4
    for i, c in volcano_locations.iterrows():
        b = great_circle((row['Latitude'], row['Longitude']), c.tolist()).kilometers
        if b < min_dist:
            min_dist = b
    return min_dist

aleutian_earthquakes['Nearest Volcano'] = aleutian_earthquakes.apply(d_to_volcano, axis=1)

In [8]:
plt.plot(aleutian_earthquakes['Nearest Volcano'], - aleutian_earthquakes.Depth, '.')
plt.xlabel('Distance to nearest volcano (km)')
plt.ylabel('Earthquake Depth (km)')
plt.title('Earthquake depth vs. distance to nearest volcano')
plt.show()

Clearly, many quakes occur at less than 75 km of depth. But of those that are deeper, there seems to be a trend that as they go deeper, they are further away from the volcano.

We could surmise from this that there are multiple reasons for these quakes. Shallow quakes occur for one reason, deep quakes for another. Let's plot these in different colors on the map.

(Also of note in the above plot: there are many earthquakes at about 35 km of depth. This is probably because at one point that was the default depth assigned for quakes due to measurement limitations.)



In [14]:
llcrnrlat = 48
urcrnrlat = 61
llcrnrlon = -180
urcrnrlon = -150

#Where are the deep earthquakes?

deep_earthquakes = aleutian_earthquakes[(aleutian_earthquakes.Longitude > llcrnrlon) & 
                             (aleutian_earthquakes.Longitude < urcrnrlon) & 
                             (aleutian_earthquakes.Latitude > llcrnrlat) & 
                             (aleutian_earthquakes.Latitude < urcrnrlat) & 
                                  (aleutian_earthquakes.Depth > 75)]

shallow_earthquakes = aleutian_earthquakes[(aleutian_earthquakes.Longitude > llcrnrlon) & 
                             (aleutian_earthquakes.Longitude < urcrnrlon) & 
                             (aleutian_earthquakes.Latitude > llcrnrlat) & 
                             (aleutian_earthquakes.Latitude < urcrnrlat) & 
                                  (aleutian_earthquakes.Depth < 75)]

In [15]:
m = Basemap(projection='mill',llcrnrlat=llcrnrlat,urcrnrlat=urcrnrlat, llcrnrlon=llcrnrlon,urcrnrlon=urcrnrlon,lat_ts=20,resolution='l')
fig = plt.figure(figsize=(12,10))


longitudes_vol = aleutian_volcanoes.Longitude.tolist()
latitudes_vol = aleutian_volcanoes.Latitude.tolist()

longitudes_eq = deep_earthquakes.Longitude.tolist()
latitudes_eq = deep_earthquakes.Latitude.tolist()

longitudes_s_eq = shallow_earthquakes.Longitude.tolist()
latitudes_s_eq = shallow_earthquakes.Latitude.tolist()

x,y = m(longitudes_vol,latitudes_vol)
a,b= m(longitudes_eq,latitudes_eq)
s,t = m(longitudes_s_eq, latitudes_s_eq)

plt.title("Aleutian Volcanoes (red) Earthquakes (green = deep, yellow = shallow)")
m.plot(x, y, "o", markersize = 5, color = 'red')
m.plot(a, b, "o", markersize = 3, color = 'green')
m.plot(s, t, "o", markersize = 3, color = 'yellow')


m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='coral',lake_color='aqua')
m.drawmapboundary()
m.drawcountries()
plt.show()

Aha! It looks like the deeper quakes (green dots) are generally to the northwest of the Aleutians, while the shallower quakes (yellow dots) are to the southeast of the volcanic chain (red dots).

Below I fit the depth of the quakes to the distance behind the volcano chain using only the deeper quakes. This gives a measurement of the subduction angle.

In [16]:
from scipy.optimize import curve_fit

In [17]:
def y(x, m, b):
    return m * x + b

popt, pcov = curve_fit(y, deep_earthquakes['Nearest Volcano'].tolist(), deep_earthquakes['Depth'].tolist())
print(popt)

In [18]:
plt.plot(deep_earthquakes['Nearest Volcano'], - deep_earthquakes.Depth, 'g.')
dist = np.arange(0, 120, 2)
dep = y(dist, - popt[0], - popt[1])
plt.plot(dist, dep)
plt.ylabel('Earthquake Depth (km)')
plt.xlabel('Distance to nearest volcano (km)')
plt.show()

In [19]:
print('Subduction angle:', np.arctan(popt[0]) * 180 / np.pi, 'degrees')
print('Depth of Pacific plate under volcanos:', popt[1], 'km')

This would indicate that the subduction angle of the Aleutian Trench is 52 degrees (close to the actual value of 45 degrees), and the depth of the Pacific plate below the Aleutian volcanos is about 78 km.

Clearly, a lot is missing from this analysis: I should have fit with the shallow earthquakes as well, taking into account the direction, not just the distance to the nearest volcano. Also, it appears that the shallow quakes are much further from the volcanos up in the Northeast of the zone. Does the subduction angle change across the chain? This could also be analyzed.

Finally, it would be interesting to see further west in the archipelago.

(Final disclaimer: I am not a geologist..)